In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# specify imports to be used in the notebook
import tensorflow as tf
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pathlib
import cv2

from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
CATEGORIES = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']

IMG_SIZE=255

DATADIR  = '../input/plant-seedlings-classification/train'
DATADIR_TEST  = '../input/plant-seedlings-classification/test'

# Defining the model

In [ ]:
# function that returns the model with custom layers appended
def define_model(width, height):
    
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    
    random_flip_model = tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")(input_model)
    random_rotation_model = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(random_flip_model)
    random_zoom_model = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(random_rotation_model)
#     random_crop_model = tf.keras.layers.experimental.preprocessing.RandomCrop(height,width)(random_zoom_model)
#     random_contrast_model = tf.keras.layers.experimental.preprocessing.RandomContrast(0.l)(random_crop_model)
#     random_height_model = tf.keras.layers.experimental.preprocessing.RandomHeight(0.2)(random_zoom_model)
#     random_width_model = tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)(random_zoom_model)
    
    
    # main model with Incpetion - ResNet - v2 layers
    # omit the top layers as we are adding custom layers
    # use transfer learning, with weights from Imagenet trained model
    main_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top = False, weights = 'imagenet')(random_zoom_model)
    
    flattened_model = tf.keras.layers.Flatten()(main_model)
    
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    
#     dense_2 = tf.keras.layers.Dense(128, activation = 'sigmoid', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)
    
#     dropout_3 = tf.keras.layers.Dropout(0.1)(dense_2)
    
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)
    
    # flatten model to get appropriate dimensions
#     flattened_model = tf.keras.layers.Flatten()(main_model)
    
#     # add custom dropout and dense layers
#     dropout_1 = tf.keras.layers.Dropout(0.1)(flattened_model)
#     dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
#     dropout_2 = tf.keras.layers.Dropout(0.1)(dense_1)
    
#     # output of model
#     output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(0.0005)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

In [ ]:
# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.h5',
        monitor = 'val_accuracy', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_accuracy', min_delta = 1e-2, patience = 10, verbose = 1,
        mode = 'max', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

# image preprocessing 

In [ ]:
# from scipy import ndimage


# def pre_process(img):
#     """
#         1. change filter to HSV
#         2. Apply mask
#         3. use morphology on masked image 
#         4. do bitwise AND to show only important edges 
#     """
#     image_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     sensitivity = 35
    
#     lower_hsv = np.array([60 - sensitivity, 100, 50])
#     upper_hsv = np.array([60 + sensitivity, 255, 255])

#     mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
#     output = cv2.bitwise_and(img, img, mask = mask)
    
#     return output

# from sklearn.cluster import KMeans


# def pre_process_kmeans(img):
#     """
#         Using KMeans as a clustering technique to segment image
#     """
#     pic = img/255  # dividing by 255 to bring the pixel values between 0 and 1
#     pic_n = pic.reshape(pic.shape[0]*pic.shape[1], pic.shape[2]) # converting image to 2D array
#     kmeans = KMeans(n_clusters=5, random_state=0).fit(pic_n)     # randomly chosen 5 clusters
#     pic2show = kmeans.cluster_centers_[kmeans.labels_]     
#     cluster_pic = pic2show.reshape(pic.shape[0], pic.shape[1], pic.shape[2])
    
#     return cluster_pic

# def pre_process_edge(img):
#     """
#         Not used. Only useful for neural nets
#     """
#     edges = cv2.Canny(img,180,180,3)
#     return edges

In [ ]:
training_data=[]

def create_training_data():
    for category in CATEGORIES:

        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                
                # Pre-processing (via function pre_process) ##
                
#                 image = pre_process(new_array)
                image = new_array
                training_data.append([image,class_num])
            except Exception as e:
                print(e)
                pass
            
create_training_data()  

In [ ]:
train_len = len(training_data)
print("Number of training samples are ",train_len)

In [ ]:
import numpy as np

X_train=[]
y_train=[]

for categories, label in training_data:
    X_train.append(categories)
    y_train.append(label)
# X = np.array(X).reshape(train_len,-1)

y_train = np.array(y_train)

y_train.shape

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, num_classes=len(CATEGORIES))
# y_test = np_utils.to_categorical(y_test, num_classes=len(CATEGORIES))

X_train = np.array(X_train).astype('float32')

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X_train, y_train = unison_shuffled_copies(X_train, y_train)

In [ ]:
print(X_train.shape)

print(y_train.shape)

In [ ]:
# define training parameters
height = IMG_SIZE
width = IMG_SIZE
num_epochs = 100
batch_size = 16

In [ ]:
# define model and start training
model = define_model(width, height)

# the actual training
model.fit(
    X_train, y_train,
    validation_split = 0.1,
    callbacks = training_callbacks(),
    epochs = num_epochs,
)

In [ ]:
testing_data =[]
def create_test_data():

    for img in os.listdir(DATADIR_TEST):
        try:
            img_array=cv2.imread(os.path.join(DATADIR_TEST,img))
            new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))

            ## Pre-processing (via function pre_process)

#             image = pre_process(new_array)
            image = new_array
            testing_data.append([image,img])
        except Exception as e:
            print(e)
            pass
create_test_data()

In [ ]:
import numpy as np

X_test=[]
y_test=[]

for categories, label in testing_data:
    X_test.append(categories)
    y_test.append(label)
# X = np.array(X).reshape(train_len,-1)

y_test = np.array(y_test)

y_test.shape

In [ ]:
y_test[0]

In [ ]:
X_test = np.array(X_test).astype('float32')

In [ ]:
X_test[0].shape

In [ ]:
model_preds = model.predict(X_test)

In [ ]:
model_preds.shape

In [ ]:
classes = []

for data in range(0, model_preds.shape[0]):
    pred_index = model_preds[data, :].argmax(axis = -1)
    classes += [CATEGORIES[pred_index]]

In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = y_test
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
output_predictions['species'] = classes
output_predictions.to_csv('output.csv', index = False)

In [ ]:
output_predictions